# Predicting Depression through social media

In [1]:
# Getting the important variables together for Spark and other python modules. 
from pyspark.context import SparkContext
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import re


sc = SparkContext.getOrCreate()


## Parsing the data

In [2]:
# The tweets are stored in CSV files, they need to be parsed for tokenization and stopword removal. 

truePositivesPath = "../../data/truePositives"

depressionTweets = sc.textFile("../input/HR_comma_sep.csv")\
         .map(lambda line: line.split(","))\
         .filter(lambda line: len(line)>1)\
         .map(lambda line: [x.encode('UTF8') for x in line])\
         .cache()
        
nondepressionTweets = sc.textFile("../input/HR_comma_sep.csv")\
         .map(lambda line: line.split(","))\
         .filter(lambda line: len(line)>1)\
         .map(lambda line: [x.encode('UTF8') for x in line])\
         .cache()

In [2]:
def make_word_cloud(mask_file_path, text):
    """This method creates a word cloud based on the word count from the given text."""
    d = path.dirname(__file__)
    mask_image = np.array(Image.open(path.join(d, mask_file_path)))
    stopwords = set(STOPWORDS)
    stopwords.add("said")

    wc = WordCloud(background_color="white", max_words=2000, mask=mask_image, stopwords=stopwords)
    # generate word cloud
    wc.generate(text)

    # store to file
    wc.to_file(path.join(d, "alice.png"))
    
    # show
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.figure()
    plt.imshow(mask_image, cmap=plt.cm.gray, interpolation='bilinear')
    plt.axis("off")
    plt.show() 

In [3]:
df = sqlContext.read\
       .format("com.databricks.spark.csv")\
       .option("header", "true")\
       .load("../../data/truePositives/*.csv")
    
    
csvdf = spark.read.option("header", "true").csv("../../data/truePositives/*.csv")


In [4]:
csvRDD = csvdf.rdd.map(list).cache()

In [5]:
dataRDD = df.rdd.map(list).cache()

In [ ]:
csvRDD.count() 
dataRDD.count() 

In [28]:
test = csvRDD.collect()

print re.sub(r'[^\x00-\x7F]+',' ',dataRDD.collect()[13][-1])

for i in test: 
    try: 
        re.sub(r'[^\x00-\x7F]+',' ',i[-1])
    except TypeError:
        print i
        
        


..for those interested in how suicide prevention modelling operates, this video from SAX has further information https://t.co/oe1yshpdFw
[u'    Call to eliminate use of restraints and seclusion in mental health facilities https://t.co/hN0GlNVrom via\u2026"', None, None]
[u'@janeburns"', None, None]
[u'Congrats Dr Seb Rosenberg! @BrainMind_Usyd\u2026 https://t.co/ia86AKSURG"', None, None]
[u'https://t.co/w0fQUD05qe"', None, None]
[u'several compelling points here to consider https://t.co/zr9qwD1GGX via @smh"', None, None]
[u'https://t.co/qSBrZzgaZR"', None, None]
[u'https://t\u2026"', None, None]
[u'https://t.co/FLgmDSTLU1"', None, None]
[u'Great article about the Great @KevinHinesStory!  #HopeHelpsHeal #RippleEffect"', None, None]
[u'piece by @seanparnell #mentalhealth #ehealth https:/\u2026"', None, None]
[u'@BrainMind_Usyd @Sydney_Uni Housing affordability: an economic', u'social &amp; mental health issue!"', None]
[u'Bearing witness for all of us to pas\u2026 https://t.co/THihM9Nplv

In [14]:
print type(test[14][-1])

<type 'unicode'>


In [ ]:
text = dataRDD.map(lambda line: [str(re.sub(r'[^\x00-\x7F]+',' ', line[-1]))])

text.take(20)